In [1]:
import pyrealsense2 as rs
import numpy as np
import cv2
import torch
import pandas as pd
import time

def get_filter(depth_colormap,variance):
    #RGB
    Rmax,Gmax,Bmax=255,255,255
    try:
        R=depth_colormap[0].mean()
        G=depth_colormap[1].mean()
        B=depth_colormap[2].mean()
        dmax=math.sqrt((Gmax-G)**2+(Bmax-B)**2)
        dmin=math.sqrt((G)**2+(B)**2)
        theta_1=math.atan(G/B)
        theta_2=math.atan((Gmax-G)/(Bmax-B))
        theta_3=math.asin((Rmax-R)/(dmax))
        theta_4=math.asin(R/dmin)
        #print(theta_1,theta_2,theta_3,theta_4)
        alpha=variance*math.cos(theta_3)
        gamma=variance*math.cos(theta_4)
        Rlow=R-variance*math.cos(theta_3)
        Glow=G-gamma*math.cos(theta_1)
        Blow=B-gamma*math.sin(theta_1)
        Rhigh=R+variance*math.sin(theta_4)
        Ghigh=G+alpha*math.sin(theta_2)
        Bhigh=B+alpha*math.cos(theta_2)
        return (int(Rhigh),int(Ghigh),int(Bhigh)),(int(Rlow),int(Glow),int(Blow))
    except :
        #print("error")
        return (18, 255, 255),(1, 190, 200)

def bincount_app(a):
    a2D = a.reshape(-1,a.shape[-1])
    col_range = (256, 256, 256) # generically : a2D.max(0)+1
    a1D = np.ravel_multi_index(a2D.T, col_range)
    return np.unravel_index(np.bincount(a1D).argmax(), col_range)

# Model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')  # or yolov5m, yolov5l, yolov5x, custom

x1=100
y1=100
light_blue = (1, 190, 200)
dark_blue = (18, 255, 255)
# Configure depth and color streams
pipeline = rs.pipeline()
config = rs.config()

# Get device product line for setting a supporting resolution
pipeline_wrapper = rs.pipeline_wrapper(pipeline)
pipeline_profile = config.resolve(pipeline_wrapper)
device = pipeline_profile.get_device()
device_product_line = str(device.get_info(rs.camera_info.product_line))

found_rgb = False
for s in device.sensors:
    if s.get_info(rs.camera_info.name) == 'RGB Camera':
        found_rgb = True
        break
if not found_rgb:
    print("The demo requires Depth camera with Color sensor")
    exit(0)

config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)

if device_product_line == 'L500':
    config.enable_stream(rs.stream.color, 960, 540, rs.format.bgr8, 30)
else:
    config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)

# Start streaming
pipeline.start(config)
try:
    while True:
        # Wait for a coherent pair of frames: depth and color
        frames = pipeline.wait_for_frames()
        depth_frame = frames.get_depth_frame()
        color_frame = frames.get_color_frame()

        if not depth_frame or not color_frame:
            continue
        # Convert images to numpy arrays
        depth_image = np.asanyarray(depth_frame.get_data())
        color_image = np.asanyarray(color_frame.get_data())
        height, width= depth_image.shape
        #print(height, width)
        depth_image = depth_image[y1:height-y1+10, x1:width-130]
        height, width = depth_image.shape
        #print(height, width)
        #break
        # Apply colormap on depth image (image must be converted to 8-bit per pixel f0.irst)
        depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(depth_image, alpha=0.03), cv2.COLORMAP_JET)

        depth_colormap_dim = depth_colormap.shape
        color_colormap_dim = color_image.shape

        # If depth and color resolutions are different, resize color image to match depth image for display
        if depth_colormap_dim != color_colormap_dim:
            depth_colormap = cv2.resize(depth_colormap, dsize=(color_colormap_dim[1], color_colormap_dim[0]), interpolation=cv2.INTER_AREA)
            hsv_real = cv2.cvtColor(depth_colormap, cv2.COLOR_RGB2HSV)
            color_max,color_min=get_filter(depth_colormap,150)
            mask = cv2.inRange(hsv_real, light_blue, dark_blue)
            image = cv2.bitwise_and(depth_colormap, depth_colormap, mask=mask)
            mask = np.where((image==(0,0,0)).all(axis=2), 0, 255).astype(np.uint8)
            result = image.copy()
            result = cv2.cvtColor(result, cv2.COLOR_BGR2BGRA)
            result[:, :, 3] = mask
            result=cv2.addWeighted(color_image,0.5,image,0.7,0)
            results = model(color_image)
            frame_obj=[]
            contours=results.pandas().xyxy[0][results.pandas().xyxy[0]['confidence']>0.6]
            for x,y,w,h,name,confidence in zip(contours['xmin'].to_numpy().tolist(),contours['ymin'].to_numpy().tolist(),contours['xmax'].to_numpy().tolist(),contours['ymax'].to_numpy().tolist(),contours['name'].to_numpy().tolist(),contours['confidence'].to_numpy().tolist()):
                #depth = depth_frame.get_distance(int(x+(w-x)/2), int(y+(h-y)/2))
                #dx ,dy, dz = rs.rs2_deproject_pixel_to_point(color_intrin, [x,y], depth)
                #distance = math.sqrt(((dx)**2) + ((dy)**2) + ((dz)**2))
                minx=int(x+((w-x)/2))-5
                miny=int(y+((h-y)/2))-5
                maxx=minx+10
                maxy=miny+10
                frame_obj=[]
                for i in range(minx,maxx):
                    column=[]
                    for j in range(miny,maxy):
                        column.append(depth_frame.get_distance(i,j))
                    frame_obj.append(column)
                dis_frame=np.array(frame_obj)
                #distance=dis_frame[dis_frame<dis_frame.mean()*2.0].mean()
                distance=dis_frame.mean()
                cv2.rectangle(color_image,(int(x),int(y)),(int(w),int(h)),(0,255,0),2)
                cv2.rectangle(color_image,(int(x),int(y+10)),(int(w),int(y)-15),(0,255,0),-1)
                cv2.rectangle(color_image,(int(x),int(y)),(int(w),int(y)-15),(0,255,0),2)
                l=int(x+((w-x)/2))
                m=int(y+((h-y)/2))
                print(x,y,w,h,l,m)
                cv2.line(color_image, (l,m), (l+20,m), (0,0,255), 2)
                cv2.line(color_image, (l,m-20), (l,m), (0,255,0), 2)
                cv2.line(color_image, (l,m), (l-5,m+5), (255,0,0), 2)
                cv2.putText(color_image," "+name+" : "+str(round(confidence,1))+", dis: "+ str(round(distance,2)),(int(x),int(y)),cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,255,255),2)
                #x line

                
            images = np.hstack((color_image,result,depth_colormap))
        else:
            images = np.hstack((color_image, depth_colormap))

        # Show images
        cv2.namedWindow('RealSense', cv2.WINDOW_AUTOSIZE)
        cv2.imshow('RealSense', images)
        cv2.waitKey(1)
        #if n>100:
        #    print("ready")
        #    cv2.imwrite('rgb_test_1.jpg', color_image)
        #    cv2.imwrite('depth_test_1.jpg', depth_colormap)
        #    break
finally:

    # Stop streaming
    pipeline.stop()

Using cache found in C:\Users\scesv7/.cache\torch\hub\ultralytics_yolov5_master

Bad key text.latex.unicode in file C:\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle, line 112 ('text.latex.unicode : False # use "ucs" and "inputenc" LaTeX packages for handling')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.3.2/matplotlibrc.template
or from the matplotlib source distribution

Bad key savefig.frameon in file C:\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle, line 423 ('savefig.frameon : True')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.3.2/matplotlibrc.template
or from the matplotlib source distribution

Bad key pgf.debug in file C:\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle, line 444 ('pgf.debug           : False')
You probably need to get an updated matplotlib

0.0 0.0 365.8249206542969 423.8264465332031 182 211
567.9625244140625 279.67230224609375 639.444580078125 368.8671875 603 324
0.0 0.0 366.9350891113281 419.7460632324219 183 209
568.1571044921875 281.5444641113281 639.1712646484375 369.8512268066406 603 325
0.0 0.0 366.40985107421875 411.35455322265625 183 205
568.0064086914062 281.5460510253906 639.2828979492188 370.5575256347656 603 326
0.0 0.0 366.79119873046875 414.9222717285156 183 207
568.1199340820312 262.38531494140625 639.7904663085938 369.2230224609375 603 315
0.0 0.0 366.026611328125 410.81939697265625 183 205
567.6672973632812 278.68658447265625 639.5484008789062 369.4842529296875 603 324
0.151580810546875 0.0 366.4535827636719 415.3298645019531 183 207
568.0628662109375 250.0721435546875 639.8973388671875 369.101318359375 603 309
462.79840087890625 373.2930603027344 640.0 479.0809020996094 551 426
0.0 0.0 366.8212585449219 415.68780517578125 183 207
567.5346069335938 278.1668395996094 639.4432983398438 369.2317199707031 60

568.4052124023438 251.82522583007812 639.7455444335938 370.1850891113281 604 311
568.3801879882812 246.03848266601562 639.8661499023438 369.5223083496094 604 307
0.0 0.0 598.3167724609375 390.9654846191406 299 195
2.559326171875 5.4378204345703125 602.9179077148438 477.586669921875 302 241
9.0899658203125 5.3131561279296875 608.4785766601562 477.2490234375 308 241
6.652740478515625 3.39093017578125 601.5494384765625 479.6732177734375 304 241
568.0106201171875 238.00296020507812 639.9776611328125 371.3789978027344 603 304
3.351806640625 0.0 554.6004638671875 385.49285888671875 278 192
0.0 0.0 395.20147705078125 360.6175537109375 197 180
568.0315551757812 259.8546142578125 639.7619018554688 369.17333984375 603 314
0.0 2.3670654296875 393.98626708984375 345.6851501464844 196 174
567.79345703125 278.85357666015625 639.4189453125 369.95526123046875 603 324
0.0 3.23370361328125 369.803955078125 352.48944091796875 184 177
567.75390625 274.014892578125 639.5985107421875 369.546142578125 603 32

377.9399719238281 387.64276123046875 636.6962890625 479.71337890625 507 433
0.0 0.0 367.27581787109375 409.953369140625 183 204
568.008056640625 259.2676086425781 639.7525634765625 368.8596496582031 603 314
376.35760498046875 387.2668762207031 636.4447631835938 479.7644958496094 506 433
0.0 0.0 366.52044677734375 410.6316223144531 183 205
568.0488891601562 257.07489013671875 639.8259887695312 368.4454345703125 603 312
374.6158447265625 387.37774658203125 636.302001953125 479.58868408203125 505 433
0.0 0.0 367.3366394042969 414.24945068359375 183 207
567.5115356445312 273.93780517578125 639.5292358398438 368.869873046875 603 321
0.0 0.0 367.18658447265625 408.6539001464844 183 204
377.9586486816406 387.54656982421875 636.4801025390625 479.7177734375 507 433
568.0297241210938 266.8905334472656 639.6744995117188 368.8273010253906 603 317
0.0 0.0 366.01422119140625 410.36090087890625 183 205
375.01953125 387.2229309082031 636.158447265625 479.7737121582031 505 433
568.0960083007812 255.168

374.5436096191406 387.6988525390625 636.529052734375 479.7069091796875 505 433
0.0 0.0 366.46759033203125 404.4051513671875 183 202
567.89013671875 281.1988830566406 639.2750244140625 370.1379089355469 603 325
0.0 0.0 366.6409912109375 407.66607666015625 183 203
375.474365234375 387.93585205078125 636.449462890625 479.70361328125 505 433
567.8248291015625 275.30517578125 639.4727783203125 369.259521484375 603 322
0.0 0.0 366.776123046875 412.1080017089844 183 206
374.1830139160156 387.5166320800781 636.35107421875 479.5212097167969 505 433
568.1215209960938 281.4217529296875 639.2382202148438 370.21044921875 603 325
374.5697937011719 387.6567687988281 637.5328369140625 479.7786560058594 506 433
0.0 0.0 366.72003173828125 405.4219970703125 183 202
567.7681274414062 275.05169677734375 639.4859008789062 369.26715087890625 603 322
0.0 0.0 365.9869079589844 404.3817138671875 182 202
374.400634765625 387.62164306640625 636.6473999023438 479.66363525390625 505 433
568.0421752929688 260.812927

...

0.0 0.0 364.01873779296875 408.4679260253906 182 204
373.8614807128906 387.1712646484375 636.4864501953125 479.48431396484375 505 433
567.961181640625 261.3164978027344 639.7279052734375 369.4485168457031 603 315
0.0 0.0 362.4815673828125 408.3003845214844 181 204
376.4739990234375 386.90533447265625 636.8187255859375 479.70867919921875 506 433
567.8953247070312 267.31103515625 639.7151489257812 368.6336669921875 603 317
0.0 0.0 363.635498046875 406.9535827636719 181 203
374.56805419921875 387.4085693359375 637.1389770507812 479.5875244140625 505 433
567.5906982421875 274.03082275390625 639.5687255859375 368.72064208984375 603 321
0.0 0.0 362.46142578125 401.5122375488281 181 200
375.2201843261719 388.011962890625 636.93896484375 479.59930419921875 506 433
567.6686401367188 271.95318603515625 639.6221313476562 369.14349365234375 603 320
0.0 0.0 363.4210510253906 404.0104064941406 181 202
374.9774475097656 387.432373046875 636.9117431640625 479.61907958984375 505 433
568.2151489257812 2

0.0 0.0 362.8587646484375 422.83148193359375 181 211
373.05029296875 386.9971008300781 636.7057495117188 479.5998229980469 504 433
568.0125732421875 262.7734375 639.7266845703125 368.6923828125 603 315
0.0 0.0 362.20849609375 423.61822509765625 181 211
375.0394287109375 387.3758544921875 637.2850952148438 479.8101806640625 506 433
567.784912109375 274.52081298828125 639.552001953125 369.0638427734375 603 321
0.0 0.0 363.1011962890625 413.51763916015625 181 206
375.6997375488281 387.5957336425781 635.9527587890625 479.6000061035156 505 433
568.2906494140625 245.79078674316406 639.8258056640625 369.5093078613281 604 307
0.0 0.0 363.1077880859375 410.728515625 181 205
375.9585876464844 387.2203369140625 636.94482421875 479.67926025390625 506 433
567.82421875 277.2200622558594 639.392822265625 369.4527282714844 603 323
0.0 0.0 363.4930419921875 408.382568359375 181 204
375.24835205078125 387.87139892578125 637.3021240234375 479.5230712890625 506 433
568.1261596679688 255.62069702148438 639

0.0 4.9283599853515625 365.3135986328125 364.78375244140625 182 184
378.286376953125 388.12164306640625 637.0352783203125 479.5677490234375 507 433
567.734375 277.46710205078125 639.5013427734375 369.504150390625 603 323
0.0 4.6696014404296875 364.796630859375 364.8179931640625 182 184
375.38983154296875 387.3001403808594 636.4645385742188 479.6249084472656 505 433
568.148193359375 246.38427734375 639.818115234375 369.8179931640625 603 308
0.0 5.469024658203125 364.3392333984375 366.4976501464844 182 185
375.87115478515625 387.7095947265625 637.7150268554688 479.54681396484375 506 433
568.08056640625 281.0909118652344 639.1895751953125 369.8730773925781 603 325
0.0 5.678497314453125 363.48211669921875 364.83953857421875 181 185
374.77642822265625 388.17108154296875 636.1193237304688 479.6597900390625 505 433
568.3886108398438 236.56771850585938 639.7903442382812 369.6249694824219 604 303
0.0 5.330230712890625 364.9756164550781 365.0325927734375 182 185
374.8663024902344 387.92395019531

0.0 0.0 367.49981689453125 386.1573181152344 183 193
375.80865478515625 387.9623718261719 636.8888549804688 479.5531311035156 506 433
567.8569946289062 270.140625 639.5983276367188 370.14678955078125 603 320
0.0 0.0 366.5108337402344 383.46795654296875 183 191
377.0216064453125 387.75347900390625 636.8455200195312 479.71136474609375 506 433
567.9114990234375 270.8978576660156 639.598388671875 369.6817932128906 603 320
0.0658416748046875 5.0008392333984375 367.355712890625 345.1044921875 183 175
375.6892395019531 387.9954528808594 636.38134765625 479.5762634277344 506 433
567.8765258789062 277.54095458984375 639.5081176757812 369.0950927734375 603 323
1.081573486328125 3.2573089599609375 367.8211975097656 346.7852783203125 184 175
375.20458984375 388.1353759765625 636.8897094726562 479.62628173828125 506 433
568.0457763671875 240.67935180664062 639.8619384765625 369.3302307128906 603 305
0.0 4.9055938720703125 366.6595764160156 350.69696044921875 183 177
377.6147766113281 388.1438293457

KeyboardInterrupt: 